In [3]:
# %pip install pandas

In [4]:
import pandas as pd
from datetime import datetime
import os

# Path to the directory containing chunk files
# folder_path = r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks"

# # List all CSV files in the folder
# csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# # Read and concatenate all CSVs into one DataFrame
# transactions_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Source files\transactions.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")
customer_df


,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...
594638,'C1753498738','M1823072687',20.53,'es_transportation',0
594639,'C650108285','M1823072687',50.73,'es_transportation',0
594640,'C123623130','M349281107',22.44,'es_fashion',0
594641,'C1499363341','M1823072687',14.46,'es_transportation',0


In [5]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','Weight']
)
merged_df

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud_x,Source,Target,Weight,typeTrans,fraud_y
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,'C1093826151','M348934600',4.55,'es_transportation',0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,'C352968107','M348934600',39.68,'es_transportation',0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,'C2054744914','M1823072687',26.89,'es_transportation',0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,'C1760612790','M348934600',17.25,'es_transportation',0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602744,179,'C1753498738','3','F','28007','M1823072687','28007','es_transportation',20.53,0,'C1753498738','M1823072687',20.53,'es_transportation',0
602745,179,'C650108285','4','F','28007','M1823072687','28007','es_transportation',50.73,0,'C650108285','M1823072687',50.73,'es_transportation',0
602746,179,'C123623130','2','F','28007','M349281107','28007','es_fashion',22.44,0,'C123623130','M349281107',22.44,'es_fashion',0
602747,179,'C1499363341','5','M','28007','M1823072687','28007','es_transportation',14.46,0,'C1499363341','M1823072687',14.46,'es_transportation',0


In [6]:
import pandas as pd
from datetime import datetime

def detect_pattern1(merged_df: pd.DataFrame) -> pd.DataFrame:
    """
    Detects pattern1: customers with low average transaction weight but high transaction frequency
    within high-volume merchants.
    
    Args:
        merged_df (pd.DataFrame): DataFrame containing 'merchant', 'customer', 'Weight' columns.

    Returns:
        pd.DataFrame: Filtered DataFrame with pattern1 matches and appended metadata.
    """
    
    # Step 1: Merchant-level transaction count
    merchant_txn_counts = merged_df.groupby('merchant')['customer'].count().reset_index(name='txn_count')
    
    # Step 2: Only keep merchants with >50K transactions
    eligible_merchants = merchant_txn_counts[merchant_txn_counts['txn_count'] > 50000]['merchant']
    
    # Step 3: Filter to eligible merchants
    df_eligible = merged_df[merged_df['merchant'].isin(eligible_merchants)]
    
    # Step 4: Average weight per customer per merchant
    avg_weight = df_eligible.groupby(['merchant', 'customer'])['Weight'].mean().reset_index()
    
    # Step 5: Compute 10th percentile weight threshold per merchant
    weight_threshold = avg_weight.groupby('merchant')['Weight'].quantile(0.10).reset_index()
    weight_threshold.rename(columns={'Weight': 'weight_thresh'}, inplace=True)
    
    # Step 6: Transaction count per customer
    txn_count_per_customer = df_eligible.groupby(['merchant', 'customer']).size().reset_index(name='txn_count')
    
    # Step 7: Compute 90th percentile txn count threshold per merchant
    txn_count_threshold = txn_count_per_customer.groupby('merchant')['txn_count'].quantile(0.90).reset_index()
    txn_count_threshold.rename(columns={'txn_count': 'txn_thresh'}, inplace=True)
    
    # Step 8: Join thresholds
    df1 = avg_weight.merge(weight_threshold, on='merchant')
    df1 = df1.merge(txn_count_per_customer, on=['merchant', 'customer'])
    df1 = df1.merge(txn_count_threshold, on='merchant')
    
    # Step 9: Apply pattern filter
    pattern1 = df1[
        (df1['Weight'] <= df1['weight_thresh']) & 
        (df1['txn_count'] >= df1['txn_thresh'])
    ].copy()
    
    # Step 10: Add metadata
    detection_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    pattern1['patternId'] = 'PatId1'
    pattern1['actionType'] = 'UPGRADE'
    pattern1['YStartTime'] = detection_time
    pattern1['detectionTime'] = detection_time
    pattern1['MerchantId'] = pattern1['merchant']
    pattern1['customerName'] = pattern1['customer']
    
    return pattern1


In [7]:
pattern1_df = detect_pattern1(merged_df)
pattern1_df
if pattern1_df.empty:
    print("No pattern1 matches found.")
else:
    print(f"Found {len(pattern1_df)} pattern1 matches. {print(pattern1_df)}")


           merchant       customer     Weight  weight_thresh  txn_count  \
93    'M1823072687'  'C1048588440'  23.297368      24.162730        152   
424   'M1823072687'  'C1214229415'  22.869493      24.162730        138   
818   'M1823072687'  'C1433843082'  24.027676      24.162730        142   
1001  'M1823072687'    'C15276068'  23.804267      24.162730        150   
1404  'M1823072687'  'C1734287915'  23.887297      24.162730        148   
1652  'M1823072687'  'C1871594117'  23.827902      24.162730        143   
1870  'M1823072687'  'C1975392482'  23.677914      24.162730        139   
1978  'M1823072687'  'C2035771335'  23.808310      24.162730        142   
2109  'M1823072687'  'C2112563194'  23.589130      24.162730        138   
2235  'M1823072687'    'C25759288'  24.084143      24.162730        140   
2684  'M1823072687'   'C500992269'  24.146711      24.162730        152   
2778  'M1823072687'   'C566598245'  24.032222      24.162730        144   
2872  'M1823072687'   'C6

In [8]:
pattern1_df

,merchant,customer,Weight,weight_thresh,txn_count,txn_thresh,patternId,actionType,YStartTime,detectionTime,MerchantId,customerName
93,'M1823072687','C1048588440',23.297368,24.162730,152,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1048588440'
424,'M1823072687','C1214229415',22.869493,24.162730,138,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1214229415'
818,'M1823072687','C1433843082',24.027676,24.162730,142,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1433843082'
1001,'M1823072687','C15276068',23.804267,24.162730,150,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C15276068'
1404,'M1823072687','C1734287915',23.887297,24.162730,148,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1734287915'
1652,'M1823072687','C1871594117',23.827902,24.162730,143,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1871594117'
1870,'M1823072687','C1975392482',23.677914,24.162730,139,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C1975392482'
1978,'M1823072687','C2035771335',23.808310,24.162730,142,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C2035771335'
2109,'M1823072687','C2112563194',23.589130,24.162730,138,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C2112563194'
2235,'M1823072687','C25759288',24.084143,24.162730,140,138.0,PatId1,UPGRADE,2025-07-04 01:24:35,2025-07-04 01:24:35,'M1823072687','C25759288'
